In [1]:
import pandas as pd
import openpyxl


In [2]:
long_rosmap = pd.read_excel('dataset_1225_long.xlsx', dtype={"projid":"string"})
long_rosmap.head()

/home/bayzid/anaconda3/envs/ad_venv_2/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,projid,study,fu_year,scaled_to,neglifeevents,negsocexchange,dcfdx,r_pd,r_stroke,cogn_ep,...,mem_trbl_tot,age_at_visit,cesdsum,r_depres,bmi,phys5itemsum,social_isolation,hypertension_cum,dm_cum,stroke_cum
0,00009121,MAP,0,ROSMAP,NaN,NaN,1.0,4.0,NaN,0.023136,...,0.0,79.969884,0.0,4.0,26.831775,1.750000,2.2,1.0,0.0,NaN
1,00009121,MAP,1,ROSMAP,NaN,NaN,1.0,4.0,NaN,-0.046917,...,0.0,81.081451,0.0,4.0,27.615143,4.041667,2.0,1.0,0.0,NaN
2,00009121,MAP,2,ROSMAP,NaN,NaN,1.0,4.0,4.0,-0.576306,...,0.0,81.612594,0.0,4.0,26.975260,0.625000,2.0,1.0,0.0,0.0
3,00009121,MAP,3,ROSMAP,4.0,1.833333,1.0,4.0,4.0,0.167115,...,0.0,82.595483,0.0,4.0,26.401319,0.458333,1.4,1.0,0.0,0.0
4,00009121,MAP,4,ROSMAP,2.0,2.000000,1.0,3.0,4.0,-0.010461,...,0.0,83.594798,0.0,4.0,35.144482,2.166667,1.8,1.0,0.0,0.0


In [3]:
long_rosmap["iid"]=long_rosmap["study"].str.replace(" ","")+long_rosmap["projid"].astype(str)
long_rosmap.head()

,projid,study,fu_year,scaled_to,neglifeevents,negsocexchange,dcfdx,r_pd,r_stroke,cogn_ep,...,age_at_visit,cesdsum,r_depres,bmi,phys5itemsum,social_isolation,hypertension_cum,dm_cum,stroke_cum,iid
0,00009121,MAP,0,ROSMAP,NaN,NaN,1.0,4.0,NaN,0.023136,...,79.969884,0.0,4.0,26.831775,1.750000,2.2,1.0,0.0,NaN,MAP00009121
1,00009121,MAP,1,ROSMAP,NaN,NaN,1.0,4.0,NaN,-0.046917,...,81.081451,0.0,4.0,27.615143,4.041667,2.0,1.0,0.0,NaN,MAP00009121
2,00009121,MAP,2,ROSMAP,NaN,NaN,1.0,4.0,4.0,-0.576306,...,81.612594,0.0,4.0,26.975260,0.625000,2.0,1.0,0.0,0.0,MAP00009121
3,00009121,MAP,3,ROSMAP,4.0,1.833333,1.0,4.0,4.0,0.167115,...,82.595483,0.0,4.0,26.401319,0.458333,1.4,1.0,0.0,0.0,MAP00009121
4,00009121,MAP,4,ROSMAP,2.0,2.000000,1.0,3.0,4.0,-0.010461,...,83.594798,0.0,4.0,35.144482,2.166667,1.8,1.0,0.0,0.0,MAP00009121


In [4]:
# keep the last fuyear only
long_rosmap = long_rosmap.sort_values('fu_year', ascending=False).drop_duplicates(['iid'])
print( long_rosmap.head() )
# check if correct - 00009121 has max fu_year of 12
print( long_rosmap.loc[long_rosmap['projid'] == '00009121'] )

         projid study  fu_year scaled_to  neglifeevents  negsocexchange  \
10271  20739782  ROS        28    ROSMAP            NaN             NaN   
10321  20741150  ROS        28    ROSMAP            NaN             NaN   
2909   10479506  ROS        28    ROSMAP            NaN             NaN   
2066   10222303  ROS        27    ROSMAP            NaN             NaN   
9714   20596265  ROS        27    ROSMAP            NaN             NaN   

       dcfdx  r_pd  r_stroke   cogn_ep  ...  age_at_visit  cesdsum  r_depres  \
10271    2.0   2.0       4.0  0.587007  ...     98.012320      0.0       4.0   
10321    4.0   2.0       4.0 -2.065039  ...     93.253936      1.0       4.0   
2909     NaN   NaN       NaN -1.069924  ...     93.880903      3.0       NaN   
2066     1.0   NaN       4.0       NaN  ...     97.174538      1.0       4.0   
9714     4.0   4.0       4.0 -3.578499  ...    101.519507      NaN       NaN   

             bmi  phys5itemsum  social_isolation  hypertension_cum  

In [5]:
import numpy as np

# take the cdfdx and threshold it
conditions = [
    long_rosmap['dcfdx'] == 1,
    ( ( long_rosmap['dcfdx'] >= 2 ) & ( long_rosmap['dcfdx'] <= 3) ),
    ( ( long_rosmap['dcfdx'] >= 4 ) & ( long_rosmap['dcfdx'] <= 5) ),
    long_rosmap['dcfdx'] > 5
]

values = [
    int(1), int(2), int(3), int(-9)
]

long_rosmap['case_control'] = np.select(conditions, values, default = -9)

In [6]:
pheno = long_rosmap[['iid', 'case_control']]
print( pheno )

               iid  case_control
10271  ROS20739782             2
10321  ROS20741150             3
2909   ROS10479506            -9
2066   ROS10222303             1
9714   ROS20596265             3
...            ...           ...
25001  MAP64145770             1
7273   ROS20141228             3
24736  MAP62502833             1
24730  MAP62404688             1
24827  MAP62774463             2

[3741 rows x 2 columns]


In [7]:
# read FID from pheno file and create pheno.txt
# !pip install pandas_plink
from os.path import join
from pandas_plink import read_plink, write_plink1_bin
from pandas_plink import get_data_folder
(bim, fam, bed) = read_plink( "ROSMAP_arrayGenotype.bed",
                             verbose=False)

print( fam.head() )

                                    fid          iid father mother gender  \
0  KronosII_P01_6.0_A01_ROS20275399.CEL  ROS20275399      0      0      2   
1  KronosII_P01_6.0_A02_ROS10442701.CEL  ROS10442701      0      0      1   
2  KronosII_P01_6.0_A03_ROS20152393.CEL  ROS20152393      0      0      2   
3  KronosII_P01_6.0_A04_ROS20626558.CEL  ROS20626558      0      0      2   
4  KronosII_P01_6.0_A05_ROS15176592.CEL  ROS15176592      0      0      1   

  trait  i  
0    -9  0  
1    -9  1  
2    -9  2  
3    -9  3  
4    -9  4  


In [8]:
# pheno = pd.merge(pheno, fam, on = 'iid' )
# print(pheno)

fam['trait_2']=  fam.iid.map(pheno.set_index('iid').case_control)
fam['trait_2'] = fam['trait_2'].fillna(-9)
print( fam.head() )
print(fam.describe())
print(fam.info())
fam['trait'] = fam['trait_2'].astype(int)
# fam['trait'] = fam['trait_2']
fam = fam.drop(['trait_2'], axis = 1)
print(fam.head())

                                    fid          iid father mother gender  \
0  KronosII_P01_6.0_A01_ROS20275399.CEL  ROS20275399      0      0      2   
1  KronosII_P01_6.0_A02_ROS10442701.CEL  ROS10442701      0      0      1   
2  KronosII_P01_6.0_A03_ROS20152393.CEL  ROS20152393      0      0      2   
3  KronosII_P01_6.0_A04_ROS20626558.CEL  ROS20626558      0      0      2   
4  KronosII_P01_6.0_A05_ROS15176592.CEL  ROS15176592      0      0      1   

  trait  i  trait_2  
0    -9  0      3.0  
1    -9  1      3.0  
2    -9  2      3.0  
3    -9  3      3.0  
4    -9  4      1.0  
                 i      trait_2
count  1708.000000  1708.000000
mean    853.500000     1.183841
std     493.201446     2.983481
min       0.000000    -9.000000
25%     426.750000     1.000000
50%     853.500000     2.000000
75%    1280.250000     3.000000
max    1707.000000     3.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708 entries, 0 to 1707
Data columns (total 8 columns):
 #   Column

In [9]:
# save plink files
# write_plink1_bin( (bim, fam, bed), "pheno_replaced_affymetrix.bed", verbose=False)

In [10]:
with open("test_output.fam", "w") as fam_file:
    for i in range(len(fam)):
        print(fam['fid'].loc[i], fam['iid'].loc[i], fam['father'].loc[i], fam['mother'].loc[i], fam['gender'].loc[i], fam['trait'].loc[i], sep=" ", file=fam_file)
#         print("family_{}".format(i+1), "sample_{}".format(i+1), "0", "0", "0", "-9",
#               sep=" ", file=fam_file)

In [11]:
fam['trait'].value_counts()

 1    613
 3    594
 2    376
-9    125
Name: trait, dtype: int64